For this analysis, we use the cells that were used for the typing, as we are sure that they have a proper spatial STA (needed for LSC measure).

In [1]:
rootF = 'C:/Users/Sarah/Documents/Myopia/Data analysis/Myopic mice/'

In [2]:
import os as os
import sys
import numpy as np
import csv
import pickle as pickle
import pandas as pd

# Plotting
import matplotlib.pyplot as plt
import matplotlib.patches as pcs
from skimage.transform import rescale, resize, downscale_local_mean
import seaborn as sns
from matplotlib.patches import Ellipse

# Import functions
from shared.functions import plot_scale_bar, load_image_as_on_MEA
from shared.LSC_functions import local_spatial_contrast
from shared.save_load import load_obj, save_obj

In [3]:
exp = 20240222

# Experiment conditions
from shared.parameters_stimulation2 import *
pixel_size_um = 3.5
pixel_per_check = 12
nb_checks = 60

# Load data

In [31]:
path_to_original_images = rootF + '/Eye model simulations/original images/'
imF = rootF + '/Eye model simulations/convolved images/'
path_to_PSFs =  rootF + '/Eye model simulations/mouse PSFs/'

##### STAs

In [32]:
cell_data = load_obj(rootF + 'MEA/exp' + str(exp) + '/cells_characterization_data_' + str(exp))

data_clusters = list(cell_data.keys())

##### Select STA

In [ ]:
results_path = rootF + '/MEA/exp' + str(exp) + '/'
    
cluster_selection = pd.read_excel(results_path + str(exp) + '_cluster_selection.xlsx', header=0)

cluster_names = cluster_selection['Cluster']
defined_STA = cluster_selection['Comments']
STA_start = cluster_selection['Spatial STA']
STA_end = cluster_selection['Sp STA END']

skip_cells = [] # very low firing rate during chirp

# Select "YES" cells with a spatial STA
selected_cells = []
which_STA = []

for i in range(len(cluster_names)):
    if cluster_names[i] not in skip_cells and defined_STA[i] != 'no STA':
        if STA_start[i] == 0:
            print (cluster_names[i], 'end')
            which_STA.append('end')
            selected_cells.append(cluster_names[i])
        else:
            print (cluster_names[i], 'start')
            which_STA.append('start')
            selected_cells.append(cluster_names[i])
            
        
print (len(selected_cells), 'good cells!')

##### Cells with defined STA

In [34]:
exp_path = rootF + '/MEA/exp' + str(exp) + '/'

cells_clustering = load_obj(exp_path + str(exp) + '_selected_cells_clustering')
cells_clustering = np.array(cells_clustering)

print (len(cells_clustering), 'cells were typed')

143 cells were typed


# Local contrast and mean intensity in blurred images

In [36]:
pred_path = rootF + '/contrast analysis/local_contrast_2sigma_liu2022_exp' + str(exp) 

if exp == 20211129:
    skip_cells = [45, 80, 188, 192, 401, 408, 422, 525, 533, 558, 561, 614, 682]
elif exp == 20211207:
    skip_cells = [33, 52, 145, 302, 340, 406, 414, 515]
elif exp == 20211208:
    skip_cells = [99,100,276,407,424,452,472,498] # also 407, 452, 472 because too small ellipses
elif exp == 20221213:
    skip_cells = [55, 180, 232, 294, 339, 365, 563, 570, 665, 680]
elif exp == 20221214:
    skip_cells = [40, 190, 332, 957]
elif exp == 20230105:
    skip_cells = [15, 96, 177, 221, 274, 320, 326, 389, 498, 558, 571, 689, 700, 791, 832, 856, 923, 928, 1018]

# Load ellipse points within the 2sigma contour
ell_path = rootF + '/contrast analysis/ellipses_data_with_weights_2sigma_' + str(exp) 

ellipse_data_2s = load_obj(ell_path + '.pkl')

if os.path.isfile(pred_path + '.pkl'):
    print ('Loading predictions')
    local_contrasts = load_obj(pred_path + '.pkl')
    mean_intensities = load_obj(rootF + '/contrast analysis/mean_intensities_2sigma_liu2022_exp' + str(exp) + '.pkl')
else:
    print ('Calculating predictions...')

    local_contrasts = np.empty((len(cells_clustering),Nimgs,NpupDiams,Nsources,NdFs))
    mean_intensities = np.empty((len(cells_clustering),Nimgs,NpupDiams,Nsources,NdFs))

    for index in range(len(cells_clustering)):
        cell_nb = cells_clustering[index]
        print (cell_nb)
        if cell_nb in selected_cells and cell_nb not in skip_cells:
            ### Get ellipse points
            if exp in [20221213, 20221214, 20230105]:
                cell_nb = str(cell_nb)
            ellipsepoints = ellipse_data_2s[cell_nb]['ellipse_points']
            weights = ellipse_data_2s[cell_nb]['weights']
            ### Local contrast for each image
            for iImg in range(Nimgs):
                for iPup in range(NpupDiams):
                    for iEcc in range(Nsources):
                        for iDf in range(NdFs):
                            df = dFs[iDf]
                            # load blurred image
                            im = load_image_as_on_MEA(imF + 'image_{}pupDiam={}_dF='.format(Imgs[iImg], pupDiams[iPup]) \
                                            + df + 'Source_{}.png'.format(sources[iEcc]))
                            
                            ellipse_weights = weights[ellipsepoints[:,1], ellipsepoints[:,0]]
                            mean_int, lsc = local_spatial_contrast(im, ellipsepoints, ellipse_weights)
                            
                            mean_intensities[index,iImg,iPup,iEcc,iDf] = mean_int
                            local_contrasts[index,iImg,iPup,iEcc,iDf] = lsc

Calculating predictions...
5
8
13
15
18
47
48
54
89
96
117
122
134
137
141
148
152
160
169
177
201
215
218
221
230
233
240
248
255
269
271
272
273
274
281
282
284
295
301
308
311
320
326
329
330
345
347
355
356
368
389
394
396
398
404
406
410
415
420
422
424
425
451
454
462
469
482
498
508
510
540
550
553
556
557
558
562
565
569
571
577
578
580
583
585
586
589
590
596
598
600
609
620
642
657
663
689
700
704
719
733
741
744
759
765
777
791
798
817
829
832
856
878
880
900
903
906
921
923
928
929
936
937
943
944
947
951
957
962
964
966
970
975
976
979
982
990
998
1005
1010
1018
1030
1039


In [37]:
### Save predictions
save_path = rootF + '/contrast analysis/' 
                

save_obj(local_contrasts, save_path + '/local_contrast_2sigma_liu2022_exp' + str(exp))

save_obj(mean_intensities, save_path + '/mean_intensities_2sigma_liu2022_exp' + str(exp))